Get environment variable
===

In [1]:
import os

NAME_sql = os.environ.get("POSTGRES_DB")
USER_sql = os.environ.get("POSTGRES_USER")
PASSWORD_sql = os.environ.get("POSTGRES_PASSWORD")
HOST_sql = os.environ.get("POSTGRES_HOST")
PORT_sql = os.environ.get("POSTGRES_PORT")

In [2]:
NAME_sql, USER_sql, PASSWORD_sql, HOST_sql, PORT_sql

('celery_db', 'user', 'password', 'db', '5432')

> set variable

In [3]:
os.environ.setdefault("my_key", "my_value")

'my_value'

> get variable

In [4]:
os.environ.get('my_key')

'my_value'

In [5]:
MONGO_USER = os.environ.get("MONGO_INITDB_ROOT_USERNAME")
MONGO_PASSWORD = os.environ.get("MONGO_INITDB_ROOT_PASSWORD")
MONGO_HOST = os.environ.get('MONGO_HOST')

In [6]:
MONGO_USER, MONGO_PASSWORD, MONGO_HOST

('root', 'example', 'mongo')

***

Sqlalchemy
===

In [3]:
from sqlalchemy import create_engine, MetaData, Table, select, and_, or_, not_, func, case
from sqlalchemy import Column, ForeignKey, Integer, String, Float, Boolean, insert, update, delete
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from pprint import pprint
import pandas as pd

DATABASE_CONNECTION_URI = f'postgresql+psycopg2://{USER_sql}:{PASSWORD_sql}@{HOST_sql}:{PORT_sql}/{NAME_sql}'
engine = create_engine(DATABASE_CONNECTION_URI)

***

In [4]:
connection = engine.connect()
metadata = MetaData()
my_user = Table('my_user', metadata, autoload=True, autoload_with=engine)

In [5]:
print(my_user.columns.keys())

['id', 'name', 'fullname', 'password']


In [6]:
pprint(repr(metadata.tables['my_user']))

("Table('my_user', MetaData(bind=None), Column('id', INTEGER(), "
 'table=<my_user>, primary_key=True, nullable=False, '
 'server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at '
 "0x7f498703b5c0>, for_update=False)), Column('name', VARCHAR(), "
 "table=<my_user>), Column('fullname', VARCHAR(), table=<my_user>), "
 "Column('password', VARCHAR(), table=<my_user>), schema=None)")


In [40]:
query = select([my_user])
query

<sqlalchemy.sql.selectable.Select at 0x7f4985f5da58; Select object>

In [41]:
ResultProxy = connection.execute(query)
ResultProxy

In [42]:
ResultSet = ResultProxy.fetchall()
ResultSet[:3]

[(1, 'Bogdan', 'Kozlovsky', '123'),
 (2, 'Leon', 'Kozlovsky', '345'),
 (3, 'Lera', 'Kozlovskay', '135')]

In [30]:
Result = connection.execute(query)
while True:
    user = Result.fetchone()
    if not user:
        break
    print("user's name is {:8s} user's fullname is {:10s}".format(user.name, user.fullname))

user's name is Bogdan   user's fullname is Kozlovsky 
user's name is Leon     user's fullname is Kozlovsky 
user's name is Lera     user's fullname is Kozlovskay
user's name is Andrey   user's fullname is Anykov    
user's name is Evgen    user's fullname is Borovskiy 
user's name is Elena    user's fullname is Gorchar   


>Big ResultSet

In [33]:
Result = connection.execute(query)
while True:
    user = Result.fetchmany(2)
    if not user:
        break
    print(user)

[(1, 'Bogdan', 'Kozlovsky', '123'), (2, 'Leon', 'Kozlovsky', '345')]
[(3, 'Lera', 'Kozlovskay', '135'), (4, 'Andrey', 'Anykov', '789')]
[(5, 'Evgen', 'Borovskiy', '0123'), (6, 'Elena', 'Gorchar', '3344')]


>format to DataFrame

In [58]:
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet) 

In [60]:
df.columns = ResultSet[0].keys()
df

,id,name,fullname,password
0,1,Bogdan,Kozlovsky,123
1,2,Leon,Kozlovsky,345
2,3,Lera,Kozlovskay,135
3,4,Andrey,Anykov,789
4,5,Evgen,Borovskiy,0123
5,6,Elena,Gorchar,3344


> Filter Data

>>Where

In [75]:
query = select([my_user.columns.name, my_user.columns.fullname]).where(my_user.columns.name == "Bogdan")
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,name,fullname
0,Bogdan,Kozlovsky


>>in

In [76]:
query = select([my_user.columns.name, my_user.columns.fullname]).where(my_user.columns.name.in_(['Bogdan', 'Leon']))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,name,fullname
0,Bogdan,Kozlovsky
1,Leon,Kozlovsky


>>and, or, not

In [87]:
query = select([my_user]).where(and_(my_user.columns.fullname == 'Kozlovsky', my_user.columns.name != 'Ermak'))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,id,name,fullname,password
0,1,Bogdan,Kozlovsky,123
1,2,Leon,Kozlovsky,345


>> order by

In [94]:
query = select([my_user]).order_by(my_user.columns.password.desc())
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,id,name,fullname,password
0,4,Andrey,Anykov,789
1,2,Leon,Kozlovsky,345
2,6,Elena,Gorchar,3344
3,3,Lera,Kozlovskay,135
4,1,Bogdan,Kozlovsky,123
5,5,Evgen,Borovskiy,0123


>function

>>sum

In [102]:
query = select([func.sum(my_user.columns.id)])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,sum_1
0,21


>>min

In [104]:
query = select([func.min(my_user.columns.password)])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,min_1
0,0123


>>max

In [105]:
query = select([func.max(my_user.columns.password)])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,max_1
0,789


>> avg

In [106]:
query = select([func.avg(my_user.columns.id)])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,avg_1
0,3.5000000000000000


>join

In [109]:
my_user = Table('my_user', metadata, autoload=True, autoload_with=engine)
address = Table('address', metadata, autoload=True, autoload_with=engine)

In [198]:
query = select([my_user.c.id,
                my_user.c.name,
                my_user.c.fullname,
                address.c.email_address])\
.where(my_user.c.id==address.c.user_id)

ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,id,name,fullname,email_address
0,1,Bogdan,Kozlovsky,qwe@rty
1,1,Bogdan,Kozlovsky,qyui@1qq
2,1,Bogdan,Kozlovsky,ghfghf@ff
3,2,Leon,Kozlovsky,qwe@rty
4,3,Lera,Kozlovskay,uuuuu@www
5,6,Elena,Gorchar,None
6,3,Lera,Kozlovskay,ggttgg@rr


>grou by

In [158]:
query = select([func.sum(my_user.c.id).label('mean_id'),
                my_user.c.name])\
.where(my_user.c.id==address.c.user_id)\
.group_by(my_user.c.name)

ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,mean_id,name
0,2,Leon
1,3,Bogdan
2,6,Lera
3,6,Elena


>distinct

In [157]:
query = select([my_user.c.name.distinct(),
                my_user.c.fullname])

ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,name,fullname
0,Bogdan,Kozlovsky
1,Lera,Kozlovskay
2,Andrey,Anykov
3,Evgen,Borovskiy
4,Leon,Kozlovsky
5,Elena,Gorchar


>joins

In [206]:
query = select([my_user.c.name, my_user.c.fullname, address.c.email_address])
query = query.select_from(my_user.join(address, my_user.c.id==address.c.user_id))


ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,name,fullname,email_address
0,Bogdan,Kozlovsky,qwe@rty
1,Bogdan,Kozlovsky,qyui@1qq
2,Bogdan,Kozlovsky,ghfghf@ff
3,Leon,Kozlovsky,qwe@rty
4,Lera,Kozlovskay,uuuuu@www
5,Elena,Gorchar,None
6,Lera,Kozlovskay,ggttgg@rr


>Creating and Inserting Data into Tables

- create table

In [6]:
metadata = MetaData()
emp = Table('emp', metadata,
              Column('Id', Integer()),
              Column('name', String(255), nullable=False),
              Column('salary', Float(), default=100.0),
              Column('active', Boolean(), default=True))
metadata.create_all(engine) #Creates the table

- insert one row

In [214]:
query = insert(emp).values(Id=1, name='Gaben', salary=60000.00, active=True) 
ResultProxy = connection.execute(query)
ResultProxy

- insert many row

In [216]:
query = insert(emp) 
values_list = [{'Id':'2', 'name':'Hideo', 'salary':80000, 'active':False},
               {'Id':'3', 'name':'Kodjima', 'salary':70000, 'active':True}]
ResultProxy = connection.execute(query,values_list)

*lets to check*

In [224]:
query = select([emp])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,Id,name,salary,active
0,1,Gaben,60000.0,True
1,2,Hideo,80000.0,False
2,3,Kodjima,70000.0,True


>Updating data in Databases

In [230]:
query = update(emp).values(active = True).where(emp.c.Id==2)
connection.execute(query)

In [231]:
query = select([emp])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,Id,name,salary,active
0,1,Gaben,60000.0,True
1,3,Kodjima,70000.0,True
2,2,Hideo,80000.0,True


>Delete Table

In [233]:
query = delete(emp).where(emp.c.Id==1)
connection.execute(query)

In [234]:
query = select([emp])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,Id,name,salary,active
0,3,Kodjima,70000.0,True
1,2,Hideo,80000.0,True


>Dropping a Table

In [7]:
emp.drop(engine)

>ORM

>>create tablses

In [8]:
Base = declarative_base()


class User(Base):
    __tablename__ = 'my_user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    addresses = relationship("Address", backref="my_user",
                             order_by="Address.id")


class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    user_id = Column(ForeignKey('my_user.id'))
    email_address = Column(String)


Base.metadata.create_all(engine)

Peewee ORM
===

In [9]:
import peewee

Redis
===

In [7]:
from redis import Redis
r = Redis(host='redis', port=6379)

In [10]:
r.get('name')

b'Pyhon'

Mongodb
===

In [11]:
from pymongo import MongoClient
from pprint import pprint
from urllib.parse import quote_plus

uri = f"mongodb://{quote_plus(MONGO_USER)}:{quote_plus(MONGO_PASSWORD)}@{quote_plus(MONGO_HOST)}"
client = MongoClient(uri)

>create database

In [13]:
mydb = client['mydatabase']
print(client.list_database_names())

['admin', 'config', 'local']


>базы данных будто бы нет(

>Create Collection

In [10]:
mycol = mydb['customers']
print(mydb.list_collection_names())

[]


>коллекции будто бы тоже нет((

In [11]:
simli_data = {'name':'Pavel'}

In [12]:
insertes_row = mycol.insert_one(simli_data)

In [13]:
insertes_row.inserted_id

ObjectId('5dead03b13d9c21d801e6e1c')

>но после добавления данных, все появляется

In [14]:
print(client.list_database_names())
print(mydb.list_collection_names())

['admin', 'config', 'local', 'mydatabase']
['customers']


>Insert Multiple Documents

In [15]:
mylist = [
  { "name": "Amy", "address": "Apple st 652"},
  { "name": "Hannah", "address": "Mountain 21"},
  { "name": "Michael", "address": "Valley 345"},
  { "name": "Sandy", "address": "Ocean blvd 2"},
  { "name": "Betty", "address": "Green Grass 1"},
  { "name": "Richard", "address": "Sky st 331"},
  { "name": "Susan", "address": "One way 98"},
  { "name": "Vicky", "address": "Yellow Garden 2"},
  { "name": "Ben", "address": "Park Lane 38"},
  { "name": "William", "address": "Central st 954"},
  { "name": "Chuck", "address": "Main Road 989"},
  { "name": "Viola", "address": "Sideway 1633"}
]

In [16]:
my_col = client.mydatabase.customers

In [17]:
inserted_list = my_col.insert_many(mylist)

In [18]:
inserted_list.inserted_ids

[ObjectId('5dead04613d9c21d801e6e1d'),
 ObjectId('5dead04613d9c21d801e6e1e'),
 ObjectId('5dead04613d9c21d801e6e1f'),
 ObjectId('5dead04613d9c21d801e6e20'),
 ObjectId('5dead04613d9c21d801e6e21'),
 ObjectId('5dead04613d9c21d801e6e22'),
 ObjectId('5dead04613d9c21d801e6e23'),
 ObjectId('5dead04613d9c21d801e6e24'),
 ObjectId('5dead04613d9c21d801e6e25'),
 ObjectId('5dead04613d9c21d801e6e26'),
 ObjectId('5dead04613d9c21d801e6e27'),
 ObjectId('5dead04613d9c21d801e6e28')]

>Find One

In [145]:
client.mydatabase.customers.find_one()

{'_id': ObjectId('5de7e8458b5e489c7af1cde2'), 'name': 'Pavel'}

>find all

In [146]:
all_row = client.mydatabase.customers.find()

In [147]:
for row in all_row:
    print(row)

{'_id': ObjectId('5de7e8458b5e489c7af1cde2'), 'name': 'Pavel'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cde3'), 'name': 'Amy', 'address': 'Apple st 652'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cde4'), 'name': 'Hannah', 'address': 'Mountain 21'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cde5'), 'name': 'Michael', 'address': 'Valley 345'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cde6'), 'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cde7'), 'name': 'Betty', 'address': 'Green Grass 1'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cde8'), 'name': 'Richard', 'address': 'Sky st 331'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cde9'), 'name': 'Susan', 'address': 'One way 98'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cdea'), 'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cdeb'), 'name': 'Ben', 'address': 'Park Lane 38'}
{'_id': ObjectId('5de7e84d8b5e489c7af1cdec'), 'name': 'William', 'address': 'Central st 954'}
{'_id': ObjectId('5de7e84d8b5e489c7

>Return Only Some Fields

In [148]:
all_row = client.mydatabase.customers.find({}, {'_id':0,'name':1, 'address':1})
for row in all_row:
    print(row)

{'name': 'Pavel'}
{'name': 'Amy', 'address': 'Apple st 652'}
{'name': 'Hannah', 'address': 'Mountain 21'}
{'name': 'Michael', 'address': 'Valley 345'}
{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Betty', 'address': 'Green Grass 1'}
{'name': 'Richard', 'address': 'Sky st 331'}
{'name': 'Susan', 'address': 'One way 98'}
{'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'name': 'Ben', 'address': 'Park Lane 38'}
{'name': 'William', 'address': 'Central st 954'}
{'name': 'Chuck', 'address': 'Main Road 989'}
{'name': 'Viola', 'address': 'Sideway 1633'}


In [149]:
all_row = client.mydatabase.customers.find({}, {'_id':0, 'address':1})
for row in all_row:
    print(row)

{}
{'address': 'Apple st 652'}
{'address': 'Mountain 21'}
{'address': 'Valley 345'}
{'address': 'Ocean blvd 2'}
{'address': 'Green Grass 1'}
{'address': 'Sky st 331'}
{'address': 'One way 98'}
{'address': 'Yellow Garden 2'}
{'address': 'Park Lane 38'}
{'address': 'Central st 954'}
{'address': 'Main Road 989'}
{'address': 'Sideway 1633'}


>Filter the Result

In [150]:
all_row = client.mydatabase.customers.find({'name':'bogdan'}, {'_id':0})
for row in all_row:
    print(row)

{'$gt':'P'}  искать значения которые "greate then" буква "P"

In [151]:
all_row = client.mydatabase.customers.find({'name':{'$gt':'P'}},{'_id':0})
for row in all_row:
    print(row)

{'name': 'Pavel'}
{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Richard', 'address': 'Sky st 331'}
{'name': 'Susan', 'address': 'One way 98'}
{'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'name': 'William', 'address': 'Central st 954'}
{'name': 'Viola', 'address': 'Sideway 1633'}


используйте регулярный выражения

In [152]:
all_row = client.mydatabase.customers.find({'name':{ "$regex": "^S" }},{'_id':0})
for row in all_row:
    print(row)

{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Susan', 'address': 'One way 98'}


>Sort

In [153]:
all_row = client.mydatabase.customers.find({},{'_id':0}).sort('name')
for row in all_row:
    print(row)

{'name': 'Amy', 'address': 'Apple st 652'}
{'name': 'Ben', 'address': 'Park Lane 38'}
{'name': 'Betty', 'address': 'Green Grass 1'}
{'name': 'Chuck', 'address': 'Main Road 989'}
{'name': 'Hannah', 'address': 'Mountain 21'}
{'name': 'Michael', 'address': 'Valley 345'}
{'name': 'Pavel'}
{'name': 'Richard', 'address': 'Sky st 331'}
{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Susan', 'address': 'One way 98'}
{'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'name': 'Viola', 'address': 'Sideway 1633'}
{'name': 'William', 'address': 'Central st 954'}


- обратный порядок

In [154]:
all_row = client.mydatabase.customers.find({},{'_id':0}).sort('name',-1)
for row in all_row:
    print(row)

{'name': 'William', 'address': 'Central st 954'}
{'name': 'Viola', 'address': 'Sideway 1633'}
{'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'name': 'Susan', 'address': 'One way 98'}
{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Richard', 'address': 'Sky st 331'}
{'name': 'Pavel'}
{'name': 'Michael', 'address': 'Valley 345'}
{'name': 'Hannah', 'address': 'Mountain 21'}
{'name': 'Chuck', 'address': 'Main Road 989'}
{'name': 'Betty', 'address': 'Green Grass 1'}
{'name': 'Ben', 'address': 'Park Lane 38'}
{'name': 'Amy', 'address': 'Apple st 652'}


>Delete

>delee one

In [155]:
collect = client.mydatabase.customers
collect.delete_one({'address': 'Apple st 652'})

In [156]:
all_row = client.mydatabase.customers.find({},{'_id':0}).sort('name',-1)
for row in all_row:
    print(row)

{'name': 'William', 'address': 'Central st 954'}
{'name': 'Viola', 'address': 'Sideway 1633'}
{'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'name': 'Susan', 'address': 'One way 98'}
{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Richard', 'address': 'Sky st 331'}
{'name': 'Pavel'}
{'name': 'Michael', 'address': 'Valley 345'}
{'name': 'Hannah', 'address': 'Mountain 21'}
{'name': 'Chuck', 'address': 'Main Road 989'}
{'name': 'Betty', 'address': 'Green Grass 1'}
{'name': 'Ben', 'address': 'Park Lane 38'}


>delete many

In [157]:
quary = {'name':{ "$regex": "l$" }}
collect = client.mydatabase.customers
collect.delete_many(quary)

In [158]:
all_row = client.mydatabase.customers.find({},{'_id':0}).sort('name',-1)
for row in all_row:
    print(row)

{'name': 'William', 'address': 'Central st 954'}
{'name': 'Viola', 'address': 'Sideway 1633'}
{'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'name': 'Susan', 'address': 'One way 98'}
{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Richard', 'address': 'Sky st 331'}
{'name': 'Hannah', 'address': 'Mountain 21'}
{'name': 'Chuck', 'address': 'Main Road 989'}
{'name': 'Betty', 'address': 'Green Grass 1'}
{'name': 'Ben', 'address': 'Park Lane 38'}


>Update Collection

In [162]:
collect = client.mydatabase.customers
myquery = { "name": "Ben" }
newvalues = { "$set": { "address": "Canyon 123" } }

collect.update_one(myquery, newvalues)

In [163]:
all_row = client.mydatabase.customers.find({},{'_id':0}).sort('name',-1)
for row in all_row:
    print(row)

{'name': 'William', 'address': 'Central st 954'}
{'name': 'Viola', 'address': 'Sideway 1633'}
{'name': 'Vicky', 'address': 'Yellow Garden 2'}
{'name': 'Susan', 'address': 'One way 98'}
{'name': 'Sandy', 'address': 'Ocean blvd 2'}
{'name': 'Richard', 'address': 'Sky st 331'}
{'name': 'Hannah', 'address': 'Mountain 21'}
{'name': 'Chuck', 'address': 'Main Road 989'}
{'name': 'Betty', 'address': 'Green Grass 1'}
{'name': 'Ben', 'address': 'Canyon 123'}


In [164]:
collect_bomus = client.mydatabase.bonuses
mylist = [
  { "name": "Amy", "bonus": 652},
  { "name": "Hannah", "bonus": 21},
  { "name": "Michael", "bonus": 345},
  { "name": "Sandy", "bonus": 2},
  { "name": "Betty", "bonus": 1},
  { "name": "Richard", "bonus": 331},
  { "name": "Susan", "bonus": 98},
  { "name": "Vicky", "bonus": 2},
  { "name": "Ben", "bonus": 38},
  { "name": "William", "bonus": 954},
  { "name": "Chuck", "bonus": 989},
  { "name": "Viola", "bonus": 1633}
]
collect_bomus.insert_many(mylist)

In [167]:
collect_bomus = client.mydatabase.bonuses
for row in collect_bomus.find({}, {'_id':0}):
    print(row)

{'name': 'Amy', 'bonus': 652}
{'name': 'Hannah', 'bonus': 21}
{'name': 'Michael', 'bonus': 345}
{'name': 'Sandy', 'bonus': 2}
{'name': 'Betty', 'bonus': 1}
{'name': 'Richard', 'bonus': 331}
{'name': 'Susan', 'bonus': 98}
{'name': 'Vicky', 'bonus': 2}
{'name': 'Ben', 'bonus': 38}
{'name': 'William', 'bonus': 954}
{'name': 'Chuck', 'bonus': 989}
{'name': 'Viola', 'bonus': 1633}


In [168]:
collect = client.mydatabase.bonuses
myquery = { "name": "Viola" }
newvalues = { "$inc": { "bonus": 67 } }

collect.update_one(myquery, newvalues)

In [169]:
collect_bomus = client.mydatabase.bonuses
for row in collect_bomus.find({}, {'_id':0}):
    print(row)

{'name': 'Amy', 'bonus': 652}
{'name': 'Hannah', 'bonus': 21}
{'name': 'Michael', 'bonus': 345}
{'name': 'Sandy', 'bonus': 2}
{'name': 'Betty', 'bonus': 1}
{'name': 'Richard', 'bonus': 331}
{'name': 'Susan', 'bonus': 98}
{'name': 'Vicky', 'bonus': 2}
{'name': 'Ben', 'bonus': 38}
{'name': 'William', 'bonus': 954}
{'name': 'Chuck', 'bonus': 989}
{'name': 'Viola', 'bonus': 1700}


>update many

In [177]:
collect = client.mydatabase.bonuses
myquery = { "name": { "$regex": "n$" } }
newvalues = { "$inc": { "bonus": 67 } }

collect.update_many(myquery, newvalues)

In [178]:
collect_bomus = client.mydatabase.bonuses
for row in collect_bomus.find({}, {'_id':0}):
    print(row)

{'name': 'Amy', 'bonus': 652}
{'name': 'Hannah', 'bonus': 21}
{'name': 'Michael', 'bonus': 345}
{'name': 'Sandy', 'bonus': 2}
{'name': 'Betty', 'bonus': 1}
{'name': 'Richard', 'bonus': 331}
{'name': 'Susan', 'bonus': 165}
{'name': 'Vicky', 'bonus': 2}
{'name': 'Ben', 'bonus': 105}
{'name': 'William', 'bonus': 954}
{'name': 'Chuck', 'bonus': 989}
{'name': 'Viola', 'bonus': 1700}


> delete all document in a collect

In [172]:
collect = client.mydatabase.customers
collect.delete_many({})

In [174]:
all_row = client.mydatabase.customers.find()
for row in all_row:
    print(row)

>Drop Collection

In [175]:
collect = client.mydatabase.customers
collect.drop()

>Limit

In [179]:
collect_bomus = client.mydatabase.bonuses
for row in collect_bomus.find({}, {'_id':0}).limit(3):
    print(row)

{'name': 'Amy', 'bonus': 652}
{'name': 'Hannah', 'bonus': 21}
{'name': 'Michael', 'bonus': 345}


Celery
===

In [1]:
from birthdays import my_sort, hello_world, add_digit, send_row4pause

In [2]:
hw = hello_world.delay()

In [3]:
hw.get()

'hello world mather!'

In [4]:
ad = add_digit.delay(4, 5)

In [5]:
ad.get()

9

In [6]:
sr = send_row4pause.delay("Bogdan", 25)

In [7]:
sr.get()

'hello Bogdan'

***

Use Celery's brocker of task
===

In [8]:
%%time
for i in range(50):
    my_sort(7)

CPU times: user 15min 21s, sys: 9.12 s, total: 15min 30s
Wall time: 15min 29s


![](One_Process.PNG)

In [6]:
%%time
done_tasks = []
for i in range(50):
    done_tasks.append(my_sort.delay(7))

print('all tasks sended')

while done_tasks:
    for dt in done_tasks:
        if dt.state != 'PENDING':
            done_tasks.remove(dt)
            
print('all tasks resolved')

all tasks sended
all tasks resolved
CPU times: user 1min 45s, sys: 46 s, total: 2min 31s
Wall time: 3min 18s


![](All_Processes.PNG)

***